In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re

In [4]:
data=pd.read_csv('raw_data.csv')
com=pd.read_csv('data.csv')
features=pd.read_csv('features.csv')

In [5]:
accuracy_dictionary={
    'SVM-Train':[],
    'SVM-Test':[],
    'RFC-Train':[],
    'RFC-Test':[],
    'Logistic-Train':[],
    'Logistic-Test':[],
    'MLP-Train':[],
    'MLP-Test':[],
    'ratio':[]
}

In [6]:
def I_label(score):
    if(score==6 or score==7):
        return 0
    if(score>=2 and score <=5):
        return 1
    else:
        return 2

In [7]:
def C_label(score):
    if(score>=4 and score<=7):
        return 0
    elif(score==2 or score==3):
        return 1
    else:
        return 2

In [8]:
def A_label(score):
    if(score>=4 and score<=6):
        return 0
    if(score>=7 and score <=9):
        return 1
    else:
        return 2

In [9]:
def AM_LC_label(score):
    if(score>=4 and score<=5):
        return 0
    elif(score>=2.6 and score<=3.9):
        return 1
    else:
        return 2

In [10]:
def rest_label(score):
    if(score>=4 and score<=6):
        return 0
    elif(score>=2.6 and score<=3.9):
        return 1
    else:
        return 2

In [110]:
df=pd.DataFrame(accuracy_dictionary)
df.to_csv('Split_Ratios accuracy.csv')

In [11]:
def EN_label(score):
    if(score>=0 and score<=25):
        return 0
    elif(score>=26 and score<=40):
        return 1
    else:
        return 2

In [12]:
def AO_label(score):
    if(score>=0 and score<=30):
        return 0
    elif(score>=31 and score<=40):
        return 1
    else:
        return 2

In [13]:
def C3_label(score):
    if(score>=36 and score<=50):
        return 0
    elif(score>=26 and score<=35):
        return 1
    else:
        return 2

In [90]:
def test1_3(trait,start,num_features,label):
    Y=[label(x) for x in list(com[trait])]
    feature_mapping=[]
    codelist=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
        x=[]
        for col in features.columns[4:]:
            val=features[col][i]
            if not pd.isna(val):
                x.append(features[col][i])
        l=len(x)
        newdict={}
        for j in range(l):
            res = re.sub(' +', ' ',x[j])
            res=res.lower()
            res=''.join(x for x in res if (x.isalpha()or x==' '))
            res=res.strip()
            newdict[res]=j
        feature_mapping.append(newdict)
    df=data[data['Code'].isin(codelist)]
    X=[]
    cnt=0
    to_delete=[]
    for column in df.columns[2:]:
        values=list(df[column])
        l=len(values)
        x=[]
        for i in range(l):
            value=values[i]
            res = re.sub(' +', ' ',value)
            res=res.lower()
            res=''.join(x for x in res if (x.isalpha()or x==' '))
            res=res.strip()
            if(res not in feature_mapping[i]):
                x=[]
                break
            else:
                x.append(feature_mapping[i][res])
        if(len(x)>0):
            X.append(x)
        else:
            to_delete.append(cnt)
        cnt=cnt+1
    new_y=[]
    for i in range(len(Y)):
        if(i not in to_delete):
            new_y.append(Y[i])
    ratio=0.5
    X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=ratio, random_state=42)
    accuracy_dictionary['ratio'].append(ratio)
    svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
    svm_classifier.fit(X_train, y_train)
    y_pred = svm_classifier.predict(X_train)
    print("Support Vector Machines: ")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['SVM-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = svm_classifier.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['SVM-Test'].append(acc)
    print("Testing accuracy: "+str(acc))
    random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    random_forest_classifier.fit(X_train, y_train)
    y_pred = random_forest_classifier.predict(X_train)
    print("Random Forest Classifier :")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['RFC-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = random_forest_classifier.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['RFC-Test'].append(acc)
    print("Testing accuracy: "+str(acc))
    logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
    logistic_regression_classifier.fit(X_train, y_train)
    y_pred = logistic_regression_classifier.predict(X_train)
    print("Logistic Regression Classifier :")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['Logistic-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = logistic_regression_classifier.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['Logistic-Test'].append(acc)
    print("Testing accuracy: "+str(acc))
    mlp = MLPClassifier(hidden_layer_sizes=(70,35), max_iter=500, random_state=42)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_train)
    print("Multi Layer Perceptron :")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['MLP-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = mlp.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['MLP-Test'].append(acc)
    print("Testing accuracy: "+str(acc))

In [91]:
def test2(trait,start,num_features,label):
    Y=[label(x) for x in list(com[trait])]
    codelist=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
    df=data[data['Code'].isin(codelist)]
    X=[]
    cnt=0
    for column in df.columns[2:]:
        values=list(df[column])
        l=len(values)
        x=[]
        for i in range(l):
            value=int(values[i])
            x.append(value)
        X.append(x)
    ratio=0.5
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=ratio, random_state=42)
    accuracy_dictionary['ratio'].append(ratio)
    svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
    svm_classifier.fit(X_train, y_train)
    y_pred = svm_classifier.predict(X_train)
    print("Support Vector Machines: ")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['SVM-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = svm_classifier.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['SVM-Test'].append(acc)
    print("Testing accuracy: "+str(acc))
    random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    random_forest_classifier.fit(X_train, y_train)
    y_pred = random_forest_classifier.predict(X_train)
    print("Random Forest Classifier :")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['RFC-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = random_forest_classifier.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['RFC-Test'].append(acc)
    print("Testing accuracy: "+str(acc))
    logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
    logistic_regression_classifier.fit(X_train, y_train)
    y_pred = logistic_regression_classifier.predict(X_train)
    print("Logistic Regression Classifier :")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['Logistic-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = logistic_regression_classifier.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['Logistic-Test'].append(acc)
    print("Testing accuracy: "+str(acc))
    mlp = MLPClassifier(hidden_layer_sizes=(70,35), max_iter=500, random_state=42)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_train)
    print("Multi Layer Perceptron :")
    acc=accuracy_score(y_train, y_pred)
    accuracy_dictionary['MLP-Train'].append(acc)
    print("Training accuracy: "+str(acc))
    y_pred = mlp.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    accuracy_dictionary['MLP-Test'].append(acc)
    print("Testing accuracy: "+str(acc))

In [92]:
test1_3('EI',0,9,I_label)

Support Vector Machines: 
Training accuracy: 0.9190497140343159
Testing accuracy: 0.8812142542894853
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.968763748350198
Logistic Regression Classifier :
Training accuracy: 0.8152221733391993
Testing accuracy: 0.8112626484821821
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9300483941926969


In [93]:
test1_3('WI',9,9,I_label)

Support Vector Machines: 
Training accuracy: 0.9073724007561437
Testing accuracy: 0.8663202645252717
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9532357109116675
Logistic Regression Classifier :
Training accuracy: 0.5675803402646503
Testing accuracy: 0.5380255077940482
Multi Layer Perceptron :
Training accuracy: 0.9995274102079396
Testing accuracy: 0.906471421823335


In [94]:
test1_3('EC',18,9,C_label)

Support Vector Machines: 
Training accuracy: 0.9158439116125999
Testing accuracy: 0.8834586466165414
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9468984962406015
Logistic Regression Classifier :
Training accuracy: 0.7898448519040903
Testing accuracy: 0.7880639097744361
Multi Layer Perceptron :
Training accuracy: 0.999059708509638
Testing accuracy: 0.8862781954887218


C:\Users\JBSCHOLOR-2020-18\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [95]:
test1_3('WC',27,9,C_label)

Support Vector Machines: 
Training accuracy: 0.9029078363725973
Testing accuracy: 0.8728437654016757
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9452932479053721
Logistic Regression Classifier :
Training accuracy: 0.7511089206505668
Testing accuracy: 0.7235091177920158
Multi Layer Perceptron :
Training accuracy: 0.9714144898965007
Testing accuracy: 0.8797437161163134


In [96]:
test1_3('EA',36,9,A_label)

Support Vector Machines: 
Training accuracy: 0.9173189823874756
Testing accuracy: 0.8811735941320293
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9657701711491442
Logistic Regression Classifier :
Training accuracy: 0.8204500978473581
Testing accuracy: 0.8176039119804401
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9046454767726161


C:\Users\JBSCHOLOR-2020-18\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [97]:
test1_3('WA',45,9,A_label)

Support Vector Machines: 
Training accuracy: 0.9224843524313914
Testing accuracy: 0.894077997111218
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9508907077515648
Logistic Regression Classifier :
Training accuracy: 0.836783822821377
Testing accuracy: 0.8334135772749157
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9041887337506018


C:\Users\JBSCHOLOR-2020-18\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [98]:
test2('AM',54,3,AM_LC_label)

Support Vector Machines: 
Training accuracy: 0.9903211614606248
Testing accuracy: 0.9841619005719313
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9951605807303123
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 1.0


In [99]:
test2('LC',57,4,AM_LC_label)

Support Vector Machines: 
Training accuracy: 0.9868015838099428
Testing accuracy: 0.9824021117465904
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9832820061592609
Logistic Regression Classifier :
Training accuracy: 0.9978002639683238
Testing accuracy: 0.9942806863176419
Multi Layer Perceptron :
Training accuracy: 0.9991201055873296
Testing accuracy: 0.9929608446986362


In [100]:
test2('MC',61,4,rest_label)

Support Vector Machines: 
Training accuracy: 0.9956005279366476
Testing accuracy: 0.989001319841619
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9788825340959085
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9995600527936648


In [101]:
test2('ND',65,2,rest_label)

Support Vector Machines: 
Training accuracy: 1.0
Testing accuracy: 1.0
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9995600527936648
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9995600527936648


In [102]:
test2('PfW',67,3,rest_label)

Support Vector Machines: 
Training accuracy: 0.9978002639683238
Testing accuracy: 0.9986801583809943
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9951605807303123
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 1.0


In [103]:
test2('SE',70,4,rest_label)

Support Vector Machines: 
Training accuracy: 0.9925208974923009
Testing accuracy: 0.9898812142542894
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9819621645402552
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 1.0


In [104]:
test1_3('E',74,10,EN_label)

Support Vector Machines: 
Training accuracy: 0.9964788732394366
Testing accuracy: 0.9828345070422535
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9454225352112676
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9933978873239436


In [105]:
test1_3('A',84,10,AO_label)

Support Vector Machines: 
Training accuracy: 0.9942806863176419
Testing accuracy: 0.9854817421909371
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9626044874615046
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 0.9956005279366476
Testing accuracy: 0.978002639683238


In [106]:
test1_3('C',94,10,C3_label)

Support Vector Machines: 
Training accuracy: 0.9925176056338029
Testing accuracy: 0.9762428508578971
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9348878134623845
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9863616366036075


In [107]:
test1_3('N',104,10,EN_label)

Support Vector Machines: 
Training accuracy: 0.9960404751429829
Testing accuracy: 0.9846018477782666
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9428068631764188
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9960404751429829


In [108]:
test1_3('O',114,10,AO_label)

Support Vector Machines: 
Training accuracy: 0.9960404751429829
Testing accuracy: 0.9929608446986362
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.9696436427628684
Logistic Regression Classifier :
Training accuracy: 1.0
Testing accuracy: 1.0
Multi Layer Perceptron :
Training accuracy: 1.0
Testing accuracy: 0.9920809502859657


In [109]:
accuracy_dictionary

{'SVM-Train': [0.922268394035688,
  0.9052244683644001,
  0.9208670671193523,
  0.9096385542168675,
  0.91875,
  0.9245585874799358,
  1.0,
  0.99120019555121,
  0.9943779027132731,
  1.0,
  0.9980444879002689,
  0.9965778538254706,
  0.9960870628515529,
  0.9973111708628697,
  0.9941320293398533,
  0.995600097775605,
  0.9975556098753361,
  0.9218921892189219,
  0.9075605434140579,
  0.9174500587544065,
  0.9041897720271103,
  0.9189850198715989,
  0.9268733072524827,
  1.0,
  0.9914741474147415,
  0.9933993399339934,
  1.0,
  0.9980748074807481,
  0.993949394939494,
  0.9964236588720771,
  0.9980748074807481,
  0.995049504950495,
  0.9964246424642464,
  0.9964246424642464,
  0.9236329352608422,
  0.9082011474856564,
  0.9150436534586971,
  0.9049295774647887,
  0.9175401816911251,
  0.9308565531475749,
  0.9978001257071024,
  0.9899434318038969,
  0.9937146448774355,
  1.0,
  0.9984286612193589,
  0.9946574481458202,
  0.9974842767295597,
  0.9981143934632307,
  0.9952845017290161,
 